## Librairies

In [ ]:
import json
import pprint
from collections import Counter
from operator import itemgetter
import csv
import matplotlib.pyplot as plt

In [ ]:
# https://github.com/drgrib/dotmap
from dotmap import DotMap

In [ ]:
import jdcal as jd
import datetime


In [ ]:
import pandas as pd

## Ourvir et inspecter le fichier

In [ ]:
file = 'geovistory_query_exports/Person_basic_view-20210217.json'
# file = 'geovistory_query_exports/Person_basic_view-1588325117126.json'

In [ ]:
with open(file) as json_file:
    data = json.load(json_file)
type(data)    

In [ ]:
[print(a) for a in data]

In [ ]:
# number of rows in the file - create a list
rows = data['rows']
len(rows)

In [ ]:
columns = data['columns']
pprint.pprint(columns)

In [ ]:
[pprint.pprint(r) for r in rows[99:101]]

In [ ]:
### create a list of DotMap dictionaries !

dm = [DotMap(r) for r in rows[103:104]]
# pprint.pprint(dm)

In [ ]:
dm[0].col_0.entity.entity_label

In [ ]:
dm_data = DotMap(data)

In [ ]:
dm_r = dm_data.rows

In [ ]:
len(dm_r)

### Gender

In [ ]:
[r.col_2.entities[0].entity_label if r.col_2 else 'missing' for r in  dm_rows[0:5]]

In [ ]:
gl = [r.col_2.entities[0].entity_label if r.col_2.entities else 'missing' for r in  dm_rows]
print(gl[101:107])
len(gl)

In [ ]:
D = dict(Counter(gl))
print(D)
len(D)

In [ ]:
plt.bar(range(len(D)), list(D.values()), align='center', width=0.7)
plt.xticks(range(len(D)), list(D.keys()))
plt.show()

## Age by category

In [ ]:
[pprint.pprint(r) for r in rows[50:100]]

In [ ]:
def get_entities_JD(entities):
    result = []
    for e in entities:
        if e.time_span.p82.julianDay:
            result.append({'pk_teen' : e.pk_entity, 'JD' : e.time_span.p82.julianDay})
        elif e.time_span.p81a.julianDay:
            result.append({'pk_teen' : e.pk_entity, 'JD' : e.time_span.p81a.julianDay})
        elif e.time_span.p82a.julianDay:
            result.append({'pk_teen' : e.pk_entity, 'JD' : e.time_span.p82a.julianDay})
        else:
            result.append(None)

    return result

def get_entities_label(entities):
    result = []
    for e in entities:
        if e.entity_label:
            result.append(e.entity_label)
        else:
            result.append(None)

    return result

In [ ]:
ageOccupationProcessetto = []
for r in dm_r: #[299:303]:
    
    ## person identifier
    pk_entity = r.col_0.entity.pk_entity

    ## births
    birthsJD = get_entities_JD(r.col_3.entities)
    
    ## occupations, join the labels    
    occupations = ','.join([e.entity_label for e in r.col_10.entities])
    
    ## occupations, list 
    occupations_l = get_entities_label(r.col_10.entities)
    
    ## enquiries, get begin date
    enquiries = get_entities_JD(r.col_11.entities)

    ## witnesses, get begin date
    witnesses =  get_entities_JD(r.col_12.entities)
                
    ageOccupationProcessetto.append([pk_entity, birthsJD, occupations_l, enquiries, witnesses])               
    
    
    


In [ ]:
pprint.pprint(ageOccupationProcessetto[50:100])

In [ ]:
### age au moment du processetto et métier
dateBirthEnquiryOccupation = []
for l in ageOccupationProcessetto: #[50:70]:
    if l[1] and l[3]:
        if l[1][0] and l[3][0]:
            d = [jd.jd2jcal((l[1][0]['JD']),0)[0:3],jd.jd2jcal((l[3][0]['JD']),0)[0:3]]
            if l[2]:
                dateBirthEnquiryOccupation.append(d + [l[2]])
            else:    
                dateBirthEnquiryOccupation.append(d )
    else:
        dateBirthEnquiryOccupation.append(None)

    

In [ ]:
ages = [(y[1][0] - y[0][0]) for y in dateBirthEnquiryOccupation if y]
print(ages[:5])

In [ ]:
s_ages = pd.Series(ages)
s_ages.describe()

In [ ]:
std = s_ages.sort_values()
type(std)

In [ ]:
# agesCount
agesCount = pd.DataFrame(s_ages, columns=['column']).groupby(['column']).size()
# print(type(agesCount))
# agesCount.index

In [ ]:
ax = agesCount.plot.bar(x='lab', y='val', rot=70, figsize=(20,5))